In [1]:
using DrWatson
@quickactivate "CompositionalMLStudy"

using ACSets
using Catlab.CategoricalAlgebra
using DataFrames

import DBInterface:
    execute

import DrWatson:
  datadir

import IPUMS:
  load_ipums_extract,
  parse_ddi

import OMOPCDMCohortCreator:
    GenerateDatabaseDetails,
    GenerateTables

import SQLite:
    DB

In [2]:
# junction table
@present SchJunct(FreeSchema) begin
    Name::AttrType
    #
    Student::Ob
    name::Attr(Student, Name)
    #
    Class::Ob
    subject::Attr(Class, Name)
    #
    Junct::Ob
    student::Hom(Junct,Student)
    class::Hom(Junct,Class)
end
@acset_type JunctionData(SchJunct, index=[:name])

jd = JunctionData{Symbol}()

df = Dict(:Fiona => [:Math, :Philosophy, :Music],
          :Gregorio => [:Cooking, :Math, :CompSci],
          :Heather => [:Gym, :Art, :Music, :Math])

foreach(keys(df)) do student
    vals = df[student]
    o_student = add_part!(jd, :Student, name=student)
    # check the value exists...
    foreach(vals) do class
        id = incident(jd, class, :subject)
        if isempty(id)
            id = add_part!(jd, :Class, subject=class)
        end
        add_part!(jd, :Junct, student=o_student, class=only(id))
    end
end

In [3]:
jd

Student,name
1,Fiona
2,Gregorio
3,Heather
Class,subject
1,Math
2,Philosophy
3,Music
4,Cooking
5,CompSci
6,Gym
